In [1]:
import mpflash
import logging

from mpflash.logger import log, set_loglevel
from mpflash.config import config

set_loglevel("INFO")
config.interactive = False

## List the connected MCUs.

This will use filters to avoid listing bluetooth devices across the various platforms



In [2]:
from mpflash.connected import list_mcus
from mpflash.list import show_mcus


# scan boards and just work with the ones that respond with understandable data
connected_mcus = list_mcus(ignore=[], include=[], bluetooth=False)
print("Connected boards:")
for mcu in connected_mcus:
    print(f"  {mcu.board}: {mcu.serialport}")

Output()

Connected boards:
  RPI_PICO_W: COM17
  PYBV11: COM24


In [3]:
show_mcus(connected_mcus, refresh=True)

Output()

                                           Connected boards                                           
┏━━━━━━┳━━━━━━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━┓
┃Serial┃Family     ┃Port ┃Board                          ┃Variant  ┃CPU        ┃Version        ┃Build┃
┡━━━━━━╇━━━━━━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━┩
│COM17 │micropython│rp2  │RPI_PICO_W                     │         │RP2040     │v1.25.0-preview│  389│
│      │           │     │Raspberry Pi Pico W with RP2040│         │           │               │     │
│COM24 │micropython│stm32│PYBV11                         │DP_THREAD│STM32F405RG│v1.25.0-preview│  389│
│      │           │     │PYBv1.1 with STM32F405RG       │         │           │               │     │
│      │           │     │Blue Norwegian feeder          │         │           │               │     │
└──────┴───────────┴─────┴───────────────────────────────┴─────────┴───────────┴───────────────┴─────┘

In [4]:
for mcu in connected_mcus:
    # run a mpremote command
    print(f"{mcu}: ")
    print(f" - {mcu.board}: {mcu.serialport}")
    # no_info to only Return the results, rather than log them
    result = mcu.run_command("ls :", no_info=True)

    # print the result
    for l in result[1]:
        print(f" - {l.rstrip()}")

MPRemoteBoard(COM17, micropython rp2, RPI_PICO_W, 1.25.0-preview): 
 - RPI_PICO_W: COM17
 - ls :
MPRemoteBoard(COM24, micropython stm32, PYBV11-DP_THREAD, 1.25.0-preview): 
 - PYBV11: COM24
 - ls :
 -           14 foo.txt
 -            0 folder_1/
 -            0 p1_meter/
 -            0 lib/
 -          628 test.py
 -          104 board_info.toml
 -            0 System Volume Information/
 -       455656 MTCPBWY-v1.gcode


In [5]:
from pathlib import Path
from turtle import up
from mpflash.flash import flash_mcu
from mpflash.common import PORT_FWTYPES, BootloaderMethod

# flash the first connected mcu
mcu = connected_mcus[0]
firmware_bin = Path("C:\\Users\\josverl\\Downloads\\firmware\\rp2\\RPI_PICO_W-v1.25.0-preview.389.uf2")
# flash the mcu
updated = flash_mcu(mcu, fw_file=firmware_bin)

if updated:
    updated.get_mcu_info()
    print(f"  {mcu.board}: {mcu.serialport} -> {mcu}")

18:18:16 | ℹ️ Entering bootloader on COM17 using methods ['mpy', 'touch1200', 'manual']

18:18:16 | ℹ️ Attempting bootloader on COM17 using 'mpremote bootloader'

Output()

Output()

18:18:18 | ℹ️ Board is in bootloader mode

18:18:18 | ℹ️ Board ID: RPI-RP2

18:18:18 | ℹ️ Copying RPI_PICO_W-v1.25.0-preview.389.uf2 to G:\.

18:18:30 | ✅ Done copying, resetting the board.

Output()

  RPI_PICO_W: COM17 -> MPRemoteBoard(COM17, micropython rp2, RPI_PICO_W, 1.25.0-preview)


In [6]:
print(f"  {mcu.board}: {mcu.serialport} -> {mcu}")

  RPI_PICO_W: COM17 -> MPRemoteBoard(COM17, micropython rp2, RPI_PICO_W, 1.25.0-preview)


In [7]:
from mpflash.download import get_firmware_list, download_firmware_files

online = get_firmware_list(ports=["rp2", "stm32"], boards=["PYBV11"], versions=["stable", "preview"], clean=True)
for fw in online:
    print(f"  {fw.board} @ {fw.version} | {fw.filename} <-- {fw.firmware}")

Output()

Output()

  PYBV11 @ v1.24.1 | PYBV11-v1.24.1.dfu <-- https://micropython.org/resources/firmware/PYBV11-20241129-v1.24.1.dfu
  PYBV11 @ v1.24.1 | PYBV11-DP-v1.24.1.dfu <-- https://micropython.org/resources/firmware/PYBV11-DP-20241129-v1.24.1.dfu
  PYBV11 @ v1.24.1 | PYBV11-DP_THREAD-v1.24.1.dfu <-- https://micropython.org/resources/firmware/PYBV11-DP_THREAD-20241129-v1.24.1.dfu
  PYBV11 @ v1.24.1 | PYBV11-NETWORK-v1.24.1.dfu <-- https://micropython.org/resources/firmware/PYBV11-NETWORK-20241129-v1.24.1.dfu
  PYBV11 @ v1.24.1 | PYBV11-THREAD-v1.24.1.dfu <-- https://micropython.org/resources/firmware/PYBV11-THREAD-20241129-v1.24.1.dfu
  PYBV11 @ v1.25.0 | PYBV11-v1.25.0-preview.393.dfu <-- https://micropython.org/resources/firmware/PYBV11-20250317-v1.25.0-preview.393.gf1018ee5c.dfu
  PYBV11 @ v1.25.0 | PYBV11-DP-v1.25.0-preview.393.dfu <-- https://micropython.org/resources/firmware/PYBV11-DP-20250317-v1.25.0-preview.393.gf1018ee5c.dfu
  PYBV11 @ v1.25.0 | PYBV11-DP_THREAD-v1.25.0-preview.393.dfu <

In [8]:
from mpflash.config import config

download_firmware_files(online, config.firmware_folder, force=False)

(0, 10)